In [ ]:
import os
os.chdir('../')
print(os.getcwd())

In [ ]:
import torch 
import torchvision
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms

In [ ]:
# Device configuration
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

# Hyper parameters
num_epochs = 5
num_classes = 2
batch_size = 1
learning_rate = 0.0001

In [ ]:
from src.dataset import single_task_dataset
from src.dataset import multi_task_dataset

In [ ]:
df = pd.read_csv('input/data.csv')

In [ ]:
df = df.sample(frac=1).reset_index(drop=True)
df.shape

In [ ]:
df_train = df.head(2500).reset_index(drop = True)
df_test = df.tail(980).reset_index(drop = True)
df_train.shape, df_test.shape

In [ ]:
train_dataset = single_task_dataset(df_train)
test_dataset = single_task_dataset(df_test)

In [ ]:
train_dataset = multi_task_dataset(df_train)
test_dataset = multi_task_dataset(df_test)

In [ ]:
# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)

In [ ]:
def conv_block(in_channels, out_channels):
    '''Convolution Block of 3x3 kernels + batch norm + maxpool of 2x2'''

    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, 5, padding=2),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
    )
    
def set_grad(model, grad):
    for param in model.parameters():
#         print(param.shape)
        param.requires_grad = grad

In [ ]:
class TaskNet(nn.Module):
    def __init__(self):
        super(TaskNet, self).__init__()

        self.conv1 = conv_block(1, 20)
        self.conv2 = conv_block(20, 40)
        self.conv3 = conv_block(40, 60)

        self.head = nn.Linear(9 * 4 * 60, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)

        # Features of shape (batch_size, 64)
        feat = x.reshape(x.size(0), -1)

        # Output
        out = self.head(feat)

        return out

In [ ]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()

        self.conv1 = conv_block(1, 20)
        self.conv2 = conv_block(20, 40)
        self.conv3 = conv_block(40, 60)
        self.fc = nn.Linear(9 * 4 * 60 * 2, 100)
        
        task_model = TaskNet().to(device)
        try:
            checkpoint = torch.load('models/adversarial_train/model.pth', map_location=torch.device('cuda:1' if torch.cuda.is_available() else 'cpu'))
            task_model.load_state_dict(checkpoint['weights'])
        except:
            print("------------Create a brand new model from scrath -------------------------")
        self.conv11 = task_model.conv1
        self.conv22 = task_model.conv2
        self.conv33 = task_model.conv3
    
       #heads
        self.y1o = nn.Linear(100,2)
        self.y2o = nn.Linear(100,2)
        self.y3o = nn.Linear(100,2)
        self.y4o = nn.Linear(100, 2)
        self.y5o = nn.Linear(100, 2)
        self.y6o = nn.Linear(100,2)
        self.y7o = nn.Linear(100, 2)
        self.y8o = nn.Linear(100, 2)
        self.y9o = nn.Linear(100, 2)
        
    def freeze(self, subject):
        set_grad(self.y1o, False)
        set_grad(self.y2o, False)
        set_grad(self.y3o, False)
        set_grad(self.y4o, False)
        set_grad(self.y5o, False)
        set_grad(self.y6o, False)
        set_grad(self.y7o, False)
        set_grad(self.y8o, False)
        set_grad(self.y9o, False)
        
        if subject==1:
            set_grad(self.y1o, True)
        if subject==2:
            set_grad(self.y2o, True)
        if subject==3:
            set_grad(self.y3o, True)
        if subject==4:
            set_grad(self.y4o, True)
        if subject==5:
            set_grad(self.y5o, True)
        if subject==6:
            set_grad(self.y6o, True)
        if subject==7:
            set_grad(self.y7o, True)
        if subject==8:
            set_grad(self.y8o, True)
        if subject==9:
            set_grad(self.y9o, True)
        
    

    def forward(self, x):
        
        x1 = self.conv1(x)
        x1 = self.conv2(x1)
        x1 = self.conv3(x1)
        
        x2 = self.conv11(x)
        x2 = self.conv22(x2)
        x2 = self.conv33(x2)
        
        

        # Features of shape (batch_size, 64)
        feat1 = x1.reshape(x1.size(0), -1)
        feat2 = x2.reshape(x2.size(0), -1)
        feat  = torch.cat((feat1, feat2),1)
        
        head = self.fc(feat)
  
        # heads
        out1 = self.y1o(head)
        out2 = self.y2o(head)
        out3 = self.y3o(head)
        out4 = self.y4o(head)
        out5 = self.y5o(head)
        out6 = self.y6o(head)
        out7 = self.y7o(head)
        out8 = self.y8o(head)
        out9 = self.y9o(head)
                
        #y5o = torch.sigmoid(self.y5o(x1)) #should be sigmoid
#         print('Losses = ',y4o, '\n')
        return out1, out2, out3, out4, out5, out6, out7, out8, out9

In [ ]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    
    model1 = ConvNet().to(device)
    model2 = TaskNet().to(device)
    
    # Loss and optimizer
    criterion1 = nn.CrossEntropyLoss()
    optimizer1 = torch.optim.Adam(model1.parameters(), lr=learning_rate)

    criterion2 = nn.CrossEntropyLoss()
    optimizer2 = torch.optim.Adam(model2.parameters(), lr=learning_rate)
    
    
    
    for i, (images, labels, subjects) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        subjects = subjects.to(device)
        
        # Forward pass 1
        outputs1 = model1(images)
        loss1 = criterion1(outputs1[subjects-1], labels)
        model1.freeze(subjects)
        model1.to(device)
        
        
        
        # Forward pass 2
        outputs2 = model2(images)
        loss2 = criterion2(outputs2, subjects)
        
        # Backward and optimize 1
        optimizer1.zero_grad()
        loss1.backward()
        optimizer1.step()
        
        # Backward and optimize 2
        optimizer2.zero_grad()
        loss2.backward()
        optimizer2.step()
        
        torch.save({'weights': model2.state_dict()},'models/adversarial_train/model.pth')
        
        if (i+1) % 1000 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss1.item()))

In [ ]:
# Test the model
model1.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels, subjects in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model1(images)
        _, predicted = torch.max(outputs[subjects-1].data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Test Accuracy {} %'.format(100 * correct / total))

# Test the model
model1.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels, subjects in test_loader:
        images = images.to(device)
        labels = subjects.to(device)
        outputs = model2(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
#         print(predicted.item(), labels.item())

print('Train Accuracy : {} %'.format(100 * correct / total))